# Installing all dependencies

In [2]:
!pip -q install langchain tiktoken
!pip install auto-gptq --extra-index-url "https://huggingface.github.io/autogptq-index/whl/cu118/"
!pip install --upgrade optimum
!pip -qqq install einops bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265

In [ ]:
# RESTART the kernel to account changes for newly installed packages
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

# Importing Libraries

In [1]:
# IMPORTING Libraries
import argparse
from typing import Any, List, Mapping, Optional
from pydantic import BaseModel, Field

from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain.output_parsers import PydanticOutputParser

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch
import json
import os
import warnings
# Ignore
warnings.filterwarnings("ignore", category=UserWarning)

# Loading model

In [2]:
# SETTING GLOBAL VARIABLES
os.environ["SAFETENSORS_FAST_GPU"] = "1"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "TheBloke/openchat_3.5-GPTQ"

In [3]:
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
gptq_config = GPTQConfig(bits=4, damp_percent=0.01, use_cuda_fp16=True, desc_act=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             use_safetensors=True,
                                             trust_remote_code=False,
                                             revision="main",
                                             quantization_config = gptq_config
                                             )

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Creating Prompt template (For structured output)

In [4]:
class ServiceData(BaseModel):
    service_intent: str = Field(description="This field stores the intent classified from the user input.")
    location: str = Field(description="This field stores the location value extracted from the user input.")

In [6]:
# PROMPTING
## output parser
### prompt for getting structured data (JSON) from LLM
template_string = """GPT4 Correct User: As an Named Entity Recognition and Intent Classification Expert, your task is to analyze questions like the following `###user_input`:

###user_input: {user_input}

You are required to perform two main tasks based on the `###user_input`:

#Task 1: Classify the `###user_input` into one of the predefined intents {intents}. IMPORTANT: If no clear match to given intents is found, categorize the intent as "Other".

#Task 2: Extract any geographical or location-related entities present in the `###user_input`. IMPORTANT: If no specific location is mentioned, label the location as "Other".

###INSTRUCTIONS: Do NOT add any clarifying information. Output MUST follow the schema below. {format_instructions}<|end_of_turn|>GPT4 Correct Assistant:"""

output_parser = PydanticOutputParser(pydantic_object=ServiceData)
main_prompt = PromptTemplate(
    template=template_string,
    input_variables=["user_input", "intents"],
    partial_variables={"format_instructions": output_parser
.get_format_instructions()})

In [7]:
print(main_prompt.format(user_input="whats latest headline from kerala?", intents=["Weather", "News"]))

GPT4 Correct User: As an Named Entity Recognition and Intent Classification Expert, your task is to analyze questions like the following `###user_input`:

###user_input: whats latest headline from kerala?

You are required to perform two main tasks based on the `###user_input`:

#Task 1: Classify the `###user_input` into one of the predefined intents ['Weather', 'News']. IMPORTANT: If no clear match to given intents is found, categorize the intent as "Other".

#Task 2: Extract any geographical or location-related entities present in the `###user_input`. IMPORTANT: If no specific location is mentioned, label the location as "Other".

###INSTRUCTIONS: Do NOT add any clarifying information. Output MUST follow the schema below. The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}


# Langchain custom LLM

In [8]:
# class inference for langchain
## so in future use it will be compactable with langchain
class OpenchatLLM(LLM):
    """
    loading Quantized openchat3.5-GPTQ model,
    into langchain.
    model_name="TheBloke/openchat_3.5-GPTQ",
    hf_url="https://huggingface.co/TheBloke/openchat_3.5-GPTQ"
    """
    device:str = "cpu"
    chatmodel:Any = None
    chatokenizer:Any = None

    # All the optional arguments
    top_p:          Optional[float] = 0.97
    top_k:          Optional[int]   = 50
    max_tokens:     Optional[int]   = 512
    temp:           Optional[float] = 0.55
    repeat_penalty: Optional[float] = 1.2

    def __init__(self, model, tokenizer, device,  **kwargs):
        super(OpenchatLLM, self).__init__()
        self.device = device
        self.chatmodel = model
        self.chatokenizer = tokenizer

    @property
    def _llm_type(self) -> str:
        return "TheBloke/openchat_3.5-GPTQ"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        params = {
            **self._get_model_default_parameters,
            **kwargs
        }
        encoded_inputs = self.chatokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.chatmodel.generate(input_ids=encoded_inputs['input_ids'],
                        attention_mask=encoded_inputs['attention_mask'], do_sample=True, **params)
        # output_without_input = outputs[:, len(encoded_inputs['input_ids'][0]):]
        response = self.chatokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        return response

    @property
    def _get_model_default_parameters(self):
        return {
            "max_new_tokens": self.max_tokens,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            "repetition_penalty": self.repeat_penalty,
            "eos_token_id":self.chatokenizer.convert_tokens_to_ids("<|end_of_turn|>"),
            "pad_token_id":self.chatokenizer.convert_tokens_to_ids("<|pad_0|>"),
        }

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "max_new_tokens": self.max_tokens,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            "repetition_penalty": self.repeat_penalty
        }

# Conversation flow interaction
* controlled generation with followup questioning

In [21]:
from ipywidgets import widgets, interact, VBox, HBox
from IPython.display import display, HTML

class DialogflowUI:
  '''UI interaction'''
  def __init__(self, llm, prompt_template, parser, n, services:list):
    self.n = n # max retry
    self.llm = llm
    self.prompt_template = prompt_template
    self.parser = parser
    self.services = services
    self.user_input = ""
    self.llm_output = ""
    self.History = {"chat1":None, "chat2":None}
    self.lock = False

    # UI elements
    # Create the interface widgets
    self.input_box = widgets.Text(
        value='',
        placeholder='Type your message here...',
        description='USER: ',
        disabled=False,
        layout={'border': '2px black', "width":"50%", "height":"50px"})

    self.output_box = widgets.Output(layout={'border': '4px solid #73AD21', "width":"60%", "height":"250px",
                                             "overflow_y":"scroll", "scroll": "auto"})

    # Button to trigger bot response
    self.send_button = widgets.Button(
        description='CHAT',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Chat',
        icon='check', # (FontAwesome names without the `fa-` prefix)
        layout={"width":"10%", "height":"30px"}
      )

  def taking_input(self, do_send):
    if do_send:
        self.user_input = self.input_box.value
        # self.output_box.append_display_data(bot_response(user_message))
        self.input_box.value = ""
        self.print_output("USER : ", self.user_input)
        if self.lock:
          response = self.chat(self.user_input, lock=self.lock)
        else:
          response = self.chat(self.user_input, lock=False)

  def print_output(self, tag, text):
    self.output_box.append_display_data(HTML(f"<p><span style='font-weight:bold;color:#04AA6D;'>{tag}</span>{text}</p>"))

  def inject_input(self, user_input):
    if self.llm_output != "":
        if self.llm_output.service_intent != "Other":
          return f'find location from `{user_input}`, and user is looking for service {self.llm_output.service_intent}.'
        elif self.llm_output.location != "Other":
          return f'find service from `{user_input}`, and location is given as {self.llm_output.location}.'
    return user_input

  def reset(self):
    self.lock = False
    self.user_input = ""
    self.llm_output = ""
    self.History = {"chat1":None, "chat2":None}
    self.output_box.clear_output()
    self.print_output("BOT  *: ", "Resetting")

  def llm_response(self, prompt, n=0):
    ## retrying for getting structured answer
    if n==self.n: # n=> no: retrys if output is not structured as expected
      print("LLL response generation failed.")
      return None, None
    try:
      output = self.llm(prompt)
      parsed_output = self.parser.parse(output.split("GPT4 Correct Assistant:")[-1])
      s, loc = parsed_output.service_intent, parsed_output.location
      return parsed_output, parsed_output.json()
    except Exception as e:
      n += 1
      return self.llm_response(prompt, n=n)

  def chat(self, user_input, lock=False):
        if user_input in ["Exit", "Bye", "Clear", "exit", "bye", "clear"]:
          # Exit conditions
          self.reset()
          return '{"service_intent":"", "location": ""}'
        if lock:
          user_input = self.inject_input(user_input)
          # print(user_input, self.user_input)
          self.user_input = user_input

        # generating output using LLM LOOP
        ## maintain a chat history for better conversation flow
        self.History['chat1'] = self.History['chat2']
        input_prompt = self.prompt_template.format(user_input=user_input, intents=self.services)
        self.History['chat2'] =  input_prompt
        if self.History['chat1'] is not None:
            input_prompt = "<|end_of_turn|>".join([self.History['chat1'], input_prompt])
        ## LLM response generation
        processed_output, raw_output = self.llm_response(input_prompt)
        self.History['chat2'] += raw_output
        service = processed_output.service_intent
        loc = processed_output.location
        self.llm_output = processed_output

        # checking for missing keys
        if service == "Other" and loc == "Other":
          midman = f"To continue, you need to pick a service from {self.services}, also need to mention a location."
          self.followup(midman)

        elif service == "Other" and loc != "Other":
          # Service available
          midman = f"can you please specify / choose a Service from {self.services}, for the given Location: `{loc}`?"
          self.followup(midman)

        elif service != "Other" and loc == "Other":
          # Location available
          midman = f"what Location you are looking for the given Service: `{service}`?"
          self.followup(midman)
        else:
          # both are available
          midman = ""
          self.print_output("BOT  : ", raw_output)
          self.lock = False
          print("PROCESSED OUTPUT: ", raw_output)
        return raw_output

  def followup(self, midman_query):
      # if the user dont provide any of the required keys, do a followup questioning conversation
      self.print_output("BOT  *: ", midman_query)
      self.lock = True

  def run(self):
    # Combine widgets in a vertical layout
    chat_interface = VBox([self.output_box, HBox([self.input_box, self.send_button])])
    self.send_button.on_click(self.taking_input)
    return chat_interface

In [28]:
services = ["News", "Weather"]

# creating object of LLM inference class
llm = OpenchatLLM(model=model, tokenizer=tokenizer, device=device)
# start the conversation flow
chatbot = DialogflowUI(llm=llm, prompt_template=main_prompt, parser=output_parser, n=5, services=services)

In [29]:
# BOT INTERACTION From here
#------------- CHAT ----------------------#
# Display the chat interface
display(chatbot.run())

PROCESSED OUTPUT:  {"service_intent": "Weather", "location": "Megalaya"}


In [ ]:
# The END